In [ ]:
from PIL import Image
import os
import numpy as np
import cv2
import matplotlib.colors as mcolors


Image Processing Helper Methods

In [ ]:
'''
Aligns then saves two images using opencv.cv2
NOTE:   The reference image is the one that is left unchanged. The target image is warped to
        fit the reference image. 
        
        Recommend having the wider angled image be the target.
'''
def align_image(reference_dir, target_dir):
    sift = cv2.SIFT_create()
    reference_img = cv2.imread(reference_dir)
    target_img = cv2.imread(target_dir)

    keypointsRef, descriptorsRef = sift.detectAndCompute(reference_img, None)
    keypointsTarget, descriptorsTarget = sift.detectAndCompute(target_img,None)
    if keypointsRef is None or keypointsTarget is None:
        raise ValueError("Failed to detect keypoints.")
    if descriptorsRef is None or descriptorsTarget is None:
        raise ValueError("Failed to compute descriptors.")
    matcher = cv2.FlannBasedMatcher({"algorithm": 1, "trees": 5}, {"checks": 50})
    matches = matcher.knnMatch(descriptorsRef, descriptorsTarget, k = 2)

    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

    pointsRef = np.float32([keypointsRef[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    pointsTarget = np.float32([keypointsTarget[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    homography, _ = cv2.findHomography(pointsTarget, pointsRef, cv2.RANSAC)
    
    aligned_img = cv2.warpPerspective(target_img, homography, (2560,2560))

    cv2.imwrite(target_dir, aligned_img)

'''
Driver method that aligns two directories of image pairs. 

NOTE:   It is assumed that the directories have been cleaned, so errors of that nature are
        not properly handled.
'''
def align_dir(reference_dir, target_dir):
    ref_list = os.listdir(reference_dir)
    tar_list = os.listdir(target_dir)
    for i in range(len(os.listdir(reference_dir))):
        ref_path = os.path.join(reference_dir, ref_list[i])
        tar_path = os.path.join(target_dir, tar_list[i])
        try:
            align_image(ref_path, tar_path)
        except Exception as e:
            print(f"Failed to align {ref_list[i]} and {tar_list[i]}: {e}")

        print(f"Aligning: {i+1}/{len(ref_list)}\t\t\t\t\t\t\t\t", end="\r")


'''
Splits an image into a grid with dimensions X by Y where grid_size = [x,y]
Saves image tiles to output_dir using the name image_name

NOTE:   Ensure images are aligned correctly before splitting the images to ensure
        the tiles are properly aligned
'''
def split_image(image, image_name, output_dir, grid_size):
    # Open the image file
    image_width, image_height = image.size

    # Calculate the size of each tile
    tile_width = image_width // grid_size[0]
    tile_height = image_height // grid_size[1]

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop through the grid and save smaller images
    for row in range(grid_size[1]):
        for col in range(grid_size[0]):
            # Calculate the coordinates of the tile
            left = col * tile_width
            upper = row * tile_height
            right = left + tile_width
            lower = upper + tile_height

            # Crop the tile from the image
            cropped_image = image.crop((left, upper, right, lower))

            # Save the cropped image
            tile_filename = f"{image_name[:-4]}_tile_{row}_{col}.jpg"
            cropped_image.save(os.path.join(output_dir, tile_filename))

    # print(f"Image successfully split into {grid_size[0]}x{grid_size[1]} grid and saved in {output_dir}.")
'''
Driver method that splits images from a directory into a grid with dimensions X by Y where grid_size = [x,y]
'''
def split_dir(raw_dir, process_dir, grid):
    raw_files = os.listdir(raw_dir)
    for i in range(len(raw_files)):
        raw_path = os.path.join(raw_dir, raw_files[i])
        try:
            with Image.open(raw_path) as img:
                split_image(img, raw_files[i], process_dir, grid)
                print(f"Splitting: {i+1}/{len(raw_files)} images into {grid[0]}x{grid[1]} grid and saved to {process_dir}\t\t\t\t\t\t\t\t", end="\r")

        except Exception as e:
            print(f"Failed to split {raw_files[i]}: {e}")
        
'''
Resizes the images found in raw_dir to be target_size and saves them to dst_dir.

TODO:   File endings are not checked, so ensure directories contain only image types or
        impliment error handling
'''
def resize_dir(raw_dir, dst_dir, target_size):
    raw_files = os.listdir(raw_dir)
    for i in range(len(raw_files)):
        raw_path = os.path.join(raw_dir, raw_files[i])
        dst_path = os.path.join(dst_dir, raw_files[i])
        if not os.path.isfile(raw_path):
            continue
        try:
            with Image.open(raw_path) as img:
                img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                img_resized.save(dst_path)
                print(f"Resizing: {i+1}/{len(raw_files)} images saved to {dst_dir}\t\t\t\t\t\t\t\t", end="\r")

        except Exception as e:
            print(f"Failed to process {raw_files[i]}: {e}, {target_size}")


'''
Converts a raw NDVI value into a grayscale value
'''
def mono(val, range, new_range):
    converted_value = (val + 1) * 127.5
    return int(round(converted_value))


'''
Gets the timestamps to match and append.

NOTE: Adjust delimiter, timestamps_pos, and extension to match your format
'''
def get_timestamps(dir,  delimiter = "%,", timestamps_pos = -1, extension = ".jpg"):
    timestamps = set()
    ts=[]
    for filename in os.listdir(dir):
        if filename.__contains__(extension):
            timestamp = filename.split(delimiter)[timestamps_pos].split(extension)[0]
            timestamps.add(timestamp)
            ts.append(timestamp)
    return timestamps, ts

'''
Generates an NDVI image map given the file path to a red and a nir image.
Saves the generated image to ndvi_path with the same timestamp as the parents.
'''
def create_ndvi_image(red_path, noir_path, ndvi_path, stamp):
    red = Image.open(red_path)
    noir = Image.open(noir_path).resize(red.size)

    red_pix = np.array(red)
    noir_pix = np.array(noir)

    mono_ndvi_pix = np.zeros(red_pix.shape)

    for x in range(len(noir_pix)):
        for y in range(len(noir_pix[x])):
            nir = int(np.average(noir_pix[x][y]))
            red = int(np.average(red_pix[x][y]))
            ndvi_value = (nir - red) / (nir + red)
            mono_ndvi_pix[x][y] = mono(ndvi_value, [-1,1], [0,255])
    mono_ndvi = Image.fromarray(mono_ndvi_pix.astype(np.uint8))
    mono_ndvi.save(ndvi_path + "mono_ndvi" + str(stamp) + ".jpg")


'''
Driver method to generate ndvi image maps from the two parent directories.
'''
def generate_ndvi_dir(red_dir, noir_dir, ndvi_dir):
    red_list = os.listdir(red_dir)
    noir_list = os.listdir(noir_dir)
    timestamps = get_timestamps(red_dir)[1]
    for i in range(len(red_list)):
        red_path = os.path.join(red_dir,red_list[i])
        noir_path = os.path.join(noir_dir, noir_list[i])
        stamp = timestamps[i]
        create_ndvi_image(red_path, noir_path, ndvi_dir, stamp)
        print(f"Generating NDVI Map: {i+1}/{len(red_list)} images saved to {ndvi_dir}\t\t\t\t\t\t\t\t", end="\r")

'''
Removes any images that do not have a pair using the timestamps.
'''
def clean_nonpairs(rgb_dir, noir_dir, delimiter = "%,", timestamps_pos = -1, extension = ".jpg"):
    timestampsRGB = get_timestamps(rgb_dir, delimiter=delimiter, timestamps_pos=timestamps_pos, extension=extension)[0]
    timestampsNOIR = get_timestamps(noir_dir, delimiter=delimiter, timestamps_pos=timestamps_pos, extension=extension)[0]

    common_timestamps = timestampsNOIR.intersection(timestampsRGB)

    for filename in os.listdir(rgb_dir):
        if filename.endswith(extension):
            timestamp = filename.split(delimiter)[timestamps_pos].split(extension)[0]
            if timestamp not in common_timestamps:
                os.remove(os.path.join(rgb_dir, filename))
                print(f"Removed {filename} from {rgb_dir}")

    for filename in os.listdir(noir_dir):
        if filename.endswith(extension):
            timestamp = filename.split(delimiter)[timestamps_pos].split(extension)[0]
            if timestamp not in common_timestamps:
                os.remove(os.path.join(noir_dir, filename))
                print(f"Removed {filename} from {noir_dir}")

'''
Converts the directory containing NoIR + IR pass images from RGB to greyscale 
'''
def make_ir_mono(src_dir, dst_dir):
    files = os.listdir(src_dir)
    for i in range(len(files)):
        raw_path = os.path.join(src_dir, files[i])
        dst_path = os.path.join(dst_dir, files[i])
        try:
            with Image.open(raw_path) as img:
                mono = img.convert('L')
                mono.save(dst_path)
        except Exception as e:
            print(f"{raw_path} ---> {dst_path}")
            print(f"Failed to apply IR-mono to {files[i]}: {e}")
        
        print(f"Making IR Mono: {i+1}/{len(files)} images saved to {dst_dir}\t\t\t\t\t\t\t\t", end="\r")


'''
Converts the directory containing RGB images to greyscale images containing only the value for the red channel
'''
def make_red_mono(src_dir, dst_dir):
    files = os.listdir(src_dir)
    for i in range(len(files)):
        raw_path = os.path.join(src_dir, files[i])
        dst_path = os.path.join(dst_dir, files[i])
        try:
            with Image.open(raw_path) as img:
                rgb = img.convert('RGB')
                red, blue, green = rgb.split()
                red.save(dst_path)
        except Exception as e:
            print(f"Failed to apply red-mono to {files[i]}: {e}")
        print(f"Making RED Mono: {i+1}/{len(files)} images saved to {dst_dir}\t\t\t\t\t\t\t\t", end="\r")


'''
Maps a greyscale value to an RGB value according to the defined gradient
'''
def map_to_gradient(value):
    """Maps a value (0-255) to a blue-red-yellow-green gradient."""
    colors = [(0, 0, 1),  # Blue (0) (-1)
              (1, 0, 0),  # Red (85) (-.33)
              (1, 1, 0),  # Yellow (.33)
              (0, 1, 0)]  # Green (1)]
    
    norm_value = value / 255.0  # Normalize the value to 0-1
    color_map = mcolors.LinearSegmentedColormap.from_list("custom_gradient", colors)
    
    return color_map(norm_value)  # Returns an RGBA tuple


'''
Generate gradient NDVI map for analysis.

NOTE: Dr. Menon recommends not using this as training input, and only for analysis
'''
def create_ndvi_gradient_image(ndvi_path, gradient_path, stamp):
    ndvi = Image.open(ndvi_path)
    ndvi_pix = np.array(ndvi)
    print(ndvi_pix.shape[0:2])
    grad_ndvi = Image.new("RGB", (ndvi_pix.shape[0:2]))
    grad_ndvi_pix = grad_ndvi.load()

    for x in range(len(ndvi_pix)):
        for y in range(len(ndvi_pix[x])):
            print(f"On pixel: [{x}, {y}]", end="\r")
            ndvi_value = ndvi_pix[x][y][0]
            color = tuple(int(c * 255) for c in map_to_gradient(ndvi_value)[:3])
            grad_ndvi_pix[x,y] = color
    grad_ndvi.save(gradient_path + "gradient_ndvi" + str(stamp) + ".jpg")
    print("grad saved")

'''
Driver method that handles the conversion of mono ndvi maps to gradient maps

TODO:   Correct timestamping for model outputs. Right now it just adds a prefix to the beginning of the file name
        so if there is already a timestamp appended to the end it works fine, otherwise won't work with pairing
        functions as currently implemented.
'''
def generate_ndvi_gradient_dir(ndvi_dir, gradient_dir):
    ndvi_list = os.listdir(ndvi_dir)
    timestamps = get_timestamps(ndvi_dir)[1]
    for i in range(len(ndvi_list)):
        ndvi_path = os.path.join(ndvi_dir,ndvi_list[i])
        stamp = i
        create_ndvi_gradient_image(ndvi_path, gradient_dir, stamp)
        print(f"Generating Gradient NDVI Map: {i+1}/{len(ndvi_list)} images saved to {gradient_dir}\t\t\t\t\t\t\t\t", end="\r")

Validtion and Analysis Set

In [ ]:
'''
Using full images resized to the input size as input. Consider splitting the image, running the model, then stitching the outputs.
'''

# Define Raw Directories
noir_raw = "validate/noir/"
rgb_raw = "validate/rgb/"
red_raw = "validate/red/"

# Define Final Processed Directories
red_mono = "validate/red/"
rgb_dir = "validate/rgb/"
nir_mono = "validate/noir/"
ndvi_mono = "validate/ndvi/"
ndvi_grad = "validate/ndvigrad/"

clean_nonpairs(noir_raw, rgb_raw)
resize_dir(rgb_raw, rgb_dir, (256,256))
resize_dir(noir_raw, nir_mono,(256,256))
make_red_mono(rgb_dir, red_mono)
generate_ndvi_dir(red_mono, nir_mono, ndvi_mono) # This takes WAY too much memory to process, so make sure to split RGB, NIR, and RED MONO before this step
generate_ndvi_gradient_dir(ndvi_mono, ndvi_grad)

Training Set

In [ ]:
'''
Resizing images to 2560x2560 from ~4k and splitting into 10x10 grid 
'''

# Define Raw Directories
noir_raw = "_raw/noir/"
rgb_raw = "_raw/rgb/"

# Define Mid-Processed Directories
rgb_mid = "rgb/"
nir_mid = "nir_mono/"
red_mid = "red_mono/"

# Define Final Processed Directories
red_mono = "split_dataset/RED_MONO/"
rgb_dir = "split_dataset/RGB/"
nir_mono = "split_dataset/NIR_MONO/"
ndvi_mono = "split_dataset/NDVI_MONO/"
ndvi_grad = "split_dataset/ndvigrad/"

clean_nonpairs(noir_raw, rgb_raw)
make_ir_mono(noir_raw, noir_raw)
resize_dir(rgb_raw, rgb_mid, (2560,2560))
resize_dir(noir_raw, nir_mid,(2560,2560))
align_dir(nir_mid, rgb_mid)
make_red_mono(rgb_mid, red_mid)
split_dir(rgb_mid, rgb_dir, [10,10])
split_dir(red_mid, red_mono,[10,10])
split_dir(nir_mid, nir_mono,[10,10])
generate_ndvi_dir(red_mono, nir_mono, ndvi_mono) # This takes WAY too much RAM to process, so make sure to split RGB, NIR, and RED MONO before this step
generate_ndvi_gradient_dir(ndvi_mono, ndvi_grad)